Importing Libraries

In [55]:
import cv2 as cv2
import os
import tensorflow as tf
import keras
import matplotlib
import numpy as np
from keras import layers, models
from keras.preprocessing.image import ImageDataGenerator


Variables

In [70]:
dataset_path = r'D:/IMAGE_FORGERY_DATASET/Dataset'
forged_path = r'D:/IMAGE_FORGERY_DATASET/Dataset/Forged'
real_path = r'D:/IMAGE_FORGERY_DATASET/Dataset/Original'
img_height, img_width = 128,128
batch_size = 32

Preprocessing Dataset

In [58]:
def load_images_from_directory_real(directory, target_size=(img_height, img_width)):
    images = []
    labels = []
    
    for filename in os.listdir(directory):
        path = os.path.join(directory, filename)
        label = 1 
        img = cv2.imread(path)
        img = cv2.resize(img, target_size)
        img = img.astype('float32') / 255.0
        images.append(img)
        labels.append(label)

    return np.array(images), np.array(labels)

real_images,real_labels = load_images_from_directory_real(real_path, target_size=(img_height, img_width))
print(real_images,real_labels)

[[[[0.67058825 0.41568628 0.21568628]
   [0.6745098  0.41960785 0.21960784]
   [0.6784314  0.42352942 0.22352941]
   ...
   [0.64705884 0.5019608  0.33333334]
   [0.64705884 0.5058824  0.34901962]
   [0.69411767 0.5529412  0.40392157]]

  [[0.6666667  0.4117647  0.21176471]
   [0.6745098  0.41960785 0.21960784]
   [0.6784314  0.42352942 0.22352941]
   ...
   [0.6627451  0.52156866 0.3529412 ]
   [0.6627451  0.5254902  0.37254903]
   [0.68235296 0.54509807 0.39215687]]

  [[0.6627451  0.40784314 0.20784314]
   [0.6666667  0.4117647  0.21176471]
   [0.6745098  0.41960785 0.21960784]
   ...
   [0.6784314  0.5372549  0.38039216]
   [0.7058824  0.5686275  0.41568628]
   [0.6862745  0.5568628  0.40784314]]

  ...

  [[0.14901961 0.28235295 0.12156863]
   [0.14901961 0.28235295 0.12156863]
   [0.09019608 0.22352941 0.06666667]
   ...
   [0.15294118 0.2509804  0.11764706]
   [0.13725491 0.23529412 0.10196079]
   [0.12941177 0.22745098 0.09411765]]

  [[0.14117648 0.27450982 0.11372549]
   [0.1

In [59]:
def load_images_from_directory_forged(directory, target_size=(img_height, img_width)):
    images = []
    labels = []
    
    for filename in os.listdir(directory):
        path = os.path.join(directory, filename)
        label = 0 
        img = cv2.imread(path)
        img = cv2.resize(img, target_size)
        img = img.astype('float32') / 255.0
        images.append(img)
        labels.append(label)

    return np.array(images), np.array(labels)

forged_images,forged_labels = load_images_from_directory_forged(forged_path, target_size=(img_height, img_width))
print(forged_images,forged_labels)

[[[[0.7882353  0.6745098  0.5921569 ]
   [0.7882353  0.6745098  0.5921569 ]
   [0.7882353  0.6745098  0.5921569 ]
   ...
   [0.6313726  0.5411765  0.4745098 ]
   [0.59607846 0.5019608  0.4509804 ]
   [0.58431375 0.48235294 0.44705883]]

  [[0.7921569  0.6784314  0.59607846]
   [0.7921569  0.6784314  0.59607846]
   [0.7921569  0.6784314  0.59607846]
   ...
   [0.6431373  0.54509807 0.47843137]
   [0.5686275  0.47058824 0.41568628]
   [0.5921569  0.49019608 0.44705883]]

  [[0.8        0.6862745  0.6039216 ]
   [0.79607844 0.68235296 0.6       ]
   [0.79607844 0.68235296 0.6       ]
   ...
   [0.5176471  0.41960785 0.3529412 ]
   [0.5254902  0.42352942 0.36078432]
   [0.6        0.5019608  0.4392157 ]]

  ...

  [[0.7176471  0.6392157  0.61960787]
   [0.654902   0.5764706  0.5568628 ]
   [0.7372549  0.65882355 0.6392157 ]
   ...
   [0.9764706  0.93333334 0.9411765 ]
   [0.98039216 0.9372549  0.94509804]
   [0.9764706  0.93333334 0.9411765 ]]

  [[0.81960785 0.7411765  0.72156864]
   [0.7

In [60]:
print(real_images.shape)
print(forged_images.shape)

(6500, 224, 224, 3)
(6500, 224, 224, 3)


In [71]:
datagen = ImageDataGenerator(rescale=1./255)

# Define your data generators
train_datagen = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',  # assuming binary classification
    subset='training'
)

Found 13000 images belonging to 2 classes.


In [ ]:
import numpy as np

images = []
labels = []

images.append(real_images)
labels.append(real_labels)

images.append(forged_images)
labels.append(forged_labels)

labels = np.concatenate(labels, axis=0)
images = np.concatenate(images, axis=0)

print(labels)
print(images.shape)

In [ ]:
dataset = list(zip(images, labels))
np.random.shuffle(dataset)
shuffled_images, shuffled_labels = zip(*dataset)

shuffled_images = np.array(shuffled_images)
shuffled_labels = np.array(shuffled_labels)

print(shuffled_labels)
print(shuffled_images.shape)

In [72]:
from keras import layers, models


def build_generator(latent_dim=100, img_height = 224, img_width = 224):
    model = models.Sequential()
    model.add(layers.Dense(128 * 16 * 16, input_dim=latent_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Reshape((16, 16, 128)))
    target_shape = (img_height // 4, img_width // 4)  # Target size after upsampling
    model.add(layers.UpSampling2D())
    model.add(layers.Conv2D(128, (3, 3), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.UpSampling2D())
    model.add(layers.Conv2D(64, (3, 3), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.UpSampling2D())
    model.add(layers.Conv2D(3, (3, 3), padding='same', activation='sigmoid'))

    # Adjust channels to 3 for RGB
    return model

def build_discriminator(img_height=128, img_width=128):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), input_shape=(img_height, img_width, 3), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.25))
    model.add(layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.25))
    model.add(layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.25))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

def build_gan_model(generator, discriminator, latent_dim=100):
    discriminator.trainable = False  # Freeze the discriminator during combined model training

    gan_input = layers.Input(shape=(latent_dim,))
    generated_image = generator(gan_input)
    print(generated_image.shape)
    gan_output = discriminator(generated_image)

    gan_model = models.Model(gan_input, gan_output)
    gan_model.compile(loss='binary_crossentropy', optimizer='adam')

    return gan_model

# Example usage


In [74]:
epochs = 50
discriminator = build_discriminator()

discriminator.fit(train_datagen, epochs=50, steps_per_epoch=len(train_datagen),batch_size=batch_size)

Epoch 1/50
407/407 [==============================] - 52s 127ms/step - loss: 0.6659 - accuracy: 0.5945
Epoch 2/50
407/407 [==============================] - 52s 127ms/step - loss: 0.6151 - accuracy: 0.6589
Epoch 3/50
407/407 [==============================] - 51s 126ms/step - loss: 0.5190 - accuracy: 0.7425
Epoch 4/50
407/407 [==============================] - 51s 126ms/step - loss: 0.4254 - accuracy: 0.8017
Epoch 5/50
407/407 [==============================] - 52s 127ms/step - loss: 0.3456 - accuracy: 0.8458
Epoch 6/50
407/407 [==============================] - 52s 128ms/step - loss: 0.2760 - accuracy: 0.8838
Epoch 7/50
407/407 [==============================] - 53s 130ms/step - loss: 0.2278 - accuracy: 0.9052
Epoch 8/50
407/407 [==============================] - 51s 125ms/step - loss: 0.2022 - accuracy: 0.9186
Epoch 9/50
407/407 [==============================] - 51s 124ms/step - loss: 0.1797 - accuracy: 0.9302
Epoch 10/50
407/407 [==============================] - 51s 124ms/step - l

KeyboardInterrupt: 

In [ ]:
gan_model.save('forgery_detection_gan_model.h5')


In [ ]:
discriminator.save('forgery_discriminator_model.h5')

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

generated_image = generator.predict(np.random.normal(0, 1, (batch_size, 100)))

plt.imshow(generated_image)


In [ ]:
discriminator.fit(images,labels,epochs = 25 , verbose = 1)